In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [5]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate() + LoggerMessageGate())
    
    x_prev = datavar(model, :x_prior, Normal{Float64})
    add_1  = constvar(model, :add_1, 1.0)
    x      = randomvar(model, :x)
    
    x_prev_add = make_node(model, +, x_prev, add_1, x)
    
    noise = constvar(model, :noise, Normal(0.0, sqrt(200.0)))
    y     = datavar(model, :y, Dirac{Float64})
    
    add_x_and_noise = make_node(model, +, x, noise, y)
    
    activate!(model)
    
    return x_prev, x, y
end

function kalman(data)
    N = length(data)
    
    x_prev, x, y = kalman_filter_graph()

    link(x_prev, getmarginal(x))
    
    update!(x_prev, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [6]:
N = 10
data = collect(1:N) + sqrt(200.0) * randn(N);

In [7]:
kalman(data)

From variable in2 of node + => Normal{Float64}(μ=-1.0, σ=100.0)
From variable out of node + => Normal{Float64}(μ=22.831721082874584, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=21.364432434190768, σ=14.002800840280099)
From variable out of node + => Normal{Float64}(μ=8.960646462273285, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=14.223944329281123, σ=9.950371902099892)
From variable out of node + => Normal{Float64}(μ=12.28483490696222, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=12.581855116592747, σ=8.137884587711596)
From variable out of node + => Normal{Float64}(μ=14.405000882550164, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=12.035373963845837, σ=7.053456158585984)
From variable out of node + => Normal{Float64}(μ=-8.681899231323902, σ=14.142135623730951)
From variable in2 of node + => Normal{Float64}(μ=6.908427112218399, σ=6.311944030978033)
From variable out of node + => No

10-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=22.364432434190768, σ=14.002800840280099)
 Normal{Float64}(μ=15.223944329281123, σ=9.950371902099892)
 Normal{Float64}(μ=13.581855116592747, σ=8.137884587711596)
 Normal{Float64}(μ=13.035373963845837, σ=7.053456158585984)
 Normal{Float64}(μ=7.908427112218399, σ=6.311944030978033)
 Normal{Float64}(μ=8.718432444756463, σ=5.763904177042351)
 Normal{Float64}(μ=3.5196295885087814, σ=5.337605126836238)
 Normal{Float64}(μ=4.335985633131721, σ=4.993761694389224)
 Normal{Float64}(μ=3.775485206698474, σ=4.708816093480111)
 Normal{Float64}(μ=3.3561647025447283, σ=4.467670516087703)